In [ ]:
import numpy as np
import os
os.environ.setdefault('PATH', '')
from collections import deque
import gym
from gym import spaces
import cv2
cv2.ocl.setUseOpenCL(False)
import matplotlib.pyplot as plt
from cpprb import PrioritizedReplayBuffer
from tensorflow.keras.layers import Flatten, Conv2D, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import Huber
import time

#################################
## wrap deepmind source code
## Source: @misc{baselines,
#  author = {Dhariwal, Prafulla and Hesse, Christopher and Klimov, Oleg and Nichol, Alex and Plappert, Matthias and Radford, Alec and Schulman, John and Sidor, Szymon and Wu, Yuhuai and Zhokhov, Peter},
#  title = {OpenAI Baselines},
#  year = {2017},
#  publisher = {GitHub},
#  journal = {GitHub repository},
#  howpublished = {\url{https://github.com/openai/baselines}},
#}
#################################

class TimeLimit(gym.Wrapper):
    def __init__(self, env, max_episode_steps=None):
        super(TimeLimit, self).__init__(env)
        self._max_episode_steps = max_episode_steps
        self._elapsed_steps = 0

    def step(self, ac):
        observation, reward, done, info = self.env.step(ac)
        self._elapsed_steps += 1
        if self._elapsed_steps >= self._max_episode_steps:
            done = True
            info['TimeLimit.truncated'] = True
        return observation, reward, done, info

    def reset(self, **kwargs):
        self._elapsed_steps = 0
        return self.env.reset(**kwargs)

class ClipActionsWrapper(gym.Wrapper):
    def step(self, action):
        import numpy as np
        action = np.nan_to_num(action)
        action = np.clip(action, self.action_space.low, self.action_space.high)
        return self.env.step(action)

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)

class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=30):
        """Sample initial states by taking random number of no-ops on reset.
        No-op is assumed to be action 0.
        """
        gym.Wrapper.__init__(self, env)
        self.noop_max = noop_max
        self.override_num_noops = None
        self.noop_action = 0
        assert env.unwrapped.get_action_meanings()[0] == 'NOOP'

    def reset(self, **kwargs):
        """ Do no-op action for a number of steps in [1, noop_max]."""
        self.env.reset(**kwargs)
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = self.unwrapped.np_random.randint(1, self.noop_max + 1) #pylint: disable=E1101
        assert noops > 0
        obs = None
        for _ in range(noops):
            obs, _, done, _ = self.env.step(self.noop_action)
            if done:
                obs = self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)

class FireResetEnv(gym.Wrapper):
    def __init__(self, env):
        """Take action on reset for environments that are fixed until firing."""
        gym.Wrapper.__init__(self, env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def reset(self, **kwargs):
        self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)

class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env):
        """Make end-of-life == end-of-episode, but only reset on true game over.
        Done by DeepMind for the DQN and co. since it helps value estimation.
        """
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done  = True

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if lives < self.lives and lives > 0:
            # for Qbert sometimes we stay in lives == 0 condition for a few frames
            # so it's important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, info

    def reset(self, **kwargs):
        """Reset only when lives are exhausted.
        This way all states are still reachable even though lives are episodic,
        and the learner need not know about any of this behind-the-scenes.
        """
        if self.was_real_done:
            obs = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _ = self.env.step(0)
        self.lives = self.env.unwrapped.ale.lives()
        return obs

class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros((2,)+env.observation_space.shape, dtype=np.uint8)
        self._skip       = skip

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2: self._obs_buffer[0] = obs
            if i == self._skip - 1: self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)

class ClipRewardEnv(gym.RewardWrapper):
    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)

    def reward(self, reward):
        """Bin reward to {+1, 0, -1} by its sign."""
        return np.sign(reward), reward # return sign of reward for learning and for displaying results


class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env, width=84, height=84, grayscale=True, dict_space_key=None):
        """
        Warp frames to 84x84 as done in the Nature paper and later work.
        If the environment uses dictionary observations, `dict_space_key` can be specified which indicates which
        observation should be warped.
        """
        super().__init__(env)
        self._width = width
        self._height = height
        self._grayscale = grayscale
        self._key = dict_space_key
        if self._grayscale:
            num_colors = 1
        else:
            num_colors = 3

        new_space = gym.spaces.Box(
            low=0,
            high=255,
            shape=(self._height, self._width, num_colors),
            dtype=np.uint8,
        )
        if self._key is None:
            original_space = self.observation_space
            self.observation_space = new_space
        else:
            original_space = self.observation_space.spaces[self._key]
            self.observation_space.spaces[self._key] = new_space
        assert original_space.dtype == np.uint8 and len(original_space.shape) == 3

    def observation(self, obs):
        if self._key is None:
            frame = obs
        else:
            frame = obs[self._key]

        if self._grayscale:
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(
            frame, (self._width, self._height), interpolation=cv2.INTER_AREA
        )
        if self._grayscale:
            frame = np.expand_dims(frame, -1)

        if self._key is None:
            obs = frame
        else:
            obs = obs.copy()
            obs[self._key] = frame
        return obs


class FrameStack(gym.Wrapper):
    def __init__(self, env, k):
        """Stack k last frames.
        Returns lazy array, which is much more memory efficient.
        See Also
        --------
        baselines.common.atari_wrappers.LazyFrames
        """
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = spaces.Box(low=0, high=255, shape=(shp[:-1] + (shp[-1] * k,)), dtype=env.observation_space.dtype)

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):
        ob, reward, done, info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        assert len(self.frames) == self.k
        return LazyFrames(list(self.frames))

class ScaledFloatFrame(gym.ObservationWrapper):
    def __init__(self, env):
        gym.ObservationWrapper.__init__(self, env)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=env.observation_space.shape, dtype=np.float32)

    def observation(self, observation):
        # careful! This undoes the memory optimization, use
        # with smaller replay buffers only.
        return np.array(observation).astype(np.float32) / 255.0

class LazyFrames(object):
    def __init__(self, frames):
        """This object ensures that common frames between the observations are only stored once.
        It exists purely to optimize memory usage which can be huge for DQN's 1M frames replay
        buffers.
        This object should only be converted to numpy array before being passed to the model.
        You'd not believe how complex the previous solution was."""
        self._frames = frames
        self._out = None

    def _force(self):
        if self._out is None:
            self._out = np.concatenate(self._frames, axis=-1)
            self._frames = None
        return self._out

    def __array__(self, dtype=None):
        out = self._force()
        if dtype is not None:
            out = out.astype(dtype)
        return out

    def __len__(self):
        return len(self._force())

    def __getitem__(self, i):
        return self._force()[i]

    def count(self):
        frames = self._force()
        return frames.shape[frames.ndim - 1]

    def frame(self, i):
        return self._force()[..., i]

def make_atari(env_id, max_episode_steps=None):
    env = gym.make(env_id)
    assert 'NoFrameskip' in env.spec.id
    env = NoopResetEnv(env, noop_max=30)
    env = MaxAndSkipEnv(env, skip=4)
    if max_episode_steps is not None:
        env = TimeLimit(env, max_episode_steps=max_episode_steps)
    return env

def wrap_deepmind(env, episode_life=True, clip_rewards=True, frame_stack=False, scale=False):
    """Configure environment for DeepMind-style Atari.
    """
    if episode_life:
        env = EpisodicLifeEnv(env)
    if 'FIRE' in env.unwrapped.get_action_meanings():
        env = FireResetEnv(env)
    env = WarpFrame(env)
    if scale:
        env = ScaledFloatFrame(env)
    if clip_rewards:
        env = ClipRewardEnv(env)
    if frame_stack:
        env = FrameStack(env, 4)
    return env

In [ ]:
env = gym.make('SpaceInvaders-v4')

In [ ]:
env = wrap_deepmind(env, frame_stack=True, scale=True)

In [ ]:
actions = env.action_space.n

In [ ]:
height, width, channels = np.array(env.reset()).shape

In [ ]:
memory_length = 10000
class DDQN():
    def __init__(self):
        # choose action random
        self.eps = 1
        self.min_eps = 0.02
        self.max_eps = 1
        self.batch_size = 32
        self.gamma = 0.99
        self.C = 1000
        self.num_episodes = 1002
        self.reward_array = np.zeros(self.num_episodes)
        self.annealed_steps = 1000000
        self.alpha = 0.01
        self.episode_no = 0
        # initialise replay memory
        self.D = PrioritizedReplayBuffer(memory_length, {'state' : {'shape': (84, 84, 4)}, 'act' : {},
        'next_state' : {'shape' : (84, 84, 4)}, 'reward' : {}, 'done' : {}})
        self.n_steps = 1
        # initialise action-value just a linear network q1
        q1 = Sequential()
        q1.add(Conv2D(32, 8, 4, activation='relu', input_shape=(height, width, channels)))
        q1.add(Conv2D(64, 4, 2, activation='relu'))
        q1.add(Conv2D(64, 3,3, activation='relu'))
        q1.add(Flatten())
        q1.add(Dense(512, activation='relu'))
        q1.add(Dense(actions))
        q1.compile(optimizer='adam', loss=Huber())
        q1.load_weights('new_prioritizedReplay_episode_1000_weights.h5')
        # initialise target network q2
        q2 = Sequential()
        q2.add(Conv2D(32, 8, 4, activation='relu', input_shape=(height, width, channels)))
        q2.add(Conv2D(64, 4, 2, activation='relu'))
        q2.add(Conv2D(64, 3,3, activation='relu'))
        q2.add(Flatten())
        q2.add(Dense(512, activation='relu'))
        q2.add(Dense(actions))
        q2.compile(optimizer='adam', loss=Huber())
        q2.set_weights(q1.get_weights())

        self.q1, self.q2 = q1, q2
        print('Models constructed')

    def choose_action(self, state):
        if self.eps > self.min_eps:
            self.eps -= (self.max_eps - self.min_eps) / self.annealed_steps 
        q_act = self.q1.predict(state.reshape(1, 84, 84, 4), batch_size=1)
        opt_act = np.argmax(q_act)
        rand_val = np.random.random()
        if (rand_val < self.eps):
            opt_act = np.random.randint(0,actions)
        return opt_act, q_act[0, opt_act]

    def execute_action(self, state, action):
        next_state, (reward, reward_act), done, _ = env.step(action)
        self.D.add(state=state, act=action, reward=reward, next_state=next_state, done=done)
        return reward_act, done, next_state, _

    def sample_memory(self, n_step):
        s = self.D.sample(self.batch_size)
        targets = np.zeros((32, 6))
        targets = self.q1.predict(np.array(s['state'][:]).reshape(32, height, width, 4), batch_size = 1)
        fut_action = self.q2.predict(np.array(s['next_state'][:]).reshape(32, height, width, 4), batch_size = 1)
        targets[np.arange(len(s['act'])), s['act'].astype('int')] = s['reward'] + (1 - s['done']) * self.gamma * np.max(fut_action)

        self.q1.train_on_batch(s['state'], targets)

        return

    def run_algorithm(self):
        for episode in range(self.num_episodes):
            n_step = 1
            state = np.array(env.reset())
            done = False
            sum_rewards = 0
            now = time.time()
            self.current_lives = 3
            print('Running episode...')
            while not done:
                action, q_act = self.choose_action(state)
                reward, done, next_state, info= self.execute_action(state, action)
                sum_rewards += reward
                self.sample_memory(n_step)
                
                n_step += 1
                self.n_steps += 1
                state = np.array(next_state)
            end = time.time()
            
            self.episode_no += 1
            if episode % 50 == 0:
                model_weights = np.array(self.q1.get_weights())
                target_model_weights = np.array(self.q2.get_weights())
                target_model_weights = 0.01 * model_weights + 0.99 * target_model_weights
                self.q2.set_weights(target_model_weights)
                self.q1.save_weights('new_prioritizedReplay_episode_'+str(episode)+'_weights.h5')
            self.reward_array[episode] = sum_rewards
            print("Episode: {} -- Survived: {:.2f} seconds -- Reward = {}".format(episode + 1, end - now, sum_rewards))

In [ ]:
rep = DDQN()

In [ ]:
rep.run_algorithm()

In [ ]:
shap = rep.reward_array.reshape((334, 3))

In [ ]:
plt.plot(np.cumsum(np.sum(shap, axis = 1)) / np.arange(len(np.sum(shap, axis = 1))))

In [ ]:
plt.ylabel('Reward')
plt.xlabel('Episode')
plt.title('Double DQN')
plt.plot(rep.reward_array)

In [ ]:
env = gym.make('SpaceInvaders-v4', render_mode = 'human')

In [ ]:
env = wrap_deepmind(env, frame_stack=True, scale=True)

In [ ]:
done = False
state = np.array(env.reset())
rep.eps = 0.1
while not done:
    predict_movement = rep.choose_action(state)[0]
    observation, reward, done, _ = env.step(predict_movement)
    state = np.array(observation)